### Naive LSTM to learn one-char to one-char mapping with all data in each batch

In [1]:
# importing required packages

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import sequence

# fix random seed for reproducibility
np.random.seed(7)

In [2]:
# define the raw dataset

alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [3]:
# create mapping of characters to integers (0-25) and the reverse 

char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

In [4]:
# prepare the dataset of input to output pairs encoded as integers

seq_length = 1
dataX = []
dataY = []

for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i: i + seq_length]
    print("seq_in :",seq_in)
    seq_out = alphabet[i + seq_length]
    print("seq_out :", seq_out)
    dataX.append([char_to_int[char] for char in seq_in])
    print('dataX :', dataX)
    dataY.append(char_to_int[seq_out])
    print('dataY :', dataY)
    print(seq_in, '->', seq_out)

seq_in : A
seq_out : B
dataX : [[0]]
dataY : [1]
A -> B
seq_in : B
seq_out : C
dataX : [[0], [1]]
dataY : [1, 2]
B -> C
seq_in : C
seq_out : D
dataX : [[0], [1], [2]]
dataY : [1, 2, 3]
C -> D
seq_in : D
seq_out : E
dataX : [[0], [1], [2], [3]]
dataY : [1, 2, 3, 4]
D -> E
seq_in : E
seq_out : F
dataX : [[0], [1], [2], [3], [4]]
dataY : [1, 2, 3, 4, 5]
E -> F
seq_in : F
seq_out : G
dataX : [[0], [1], [2], [3], [4], [5]]
dataY : [1, 2, 3, 4, 5, 6]
F -> G
seq_in : G
seq_out : H
dataX : [[0], [1], [2], [3], [4], [5], [6]]
dataY : [1, 2, 3, 4, 5, 6, 7]
G -> H
seq_in : H
seq_out : I
dataX : [[0], [1], [2], [3], [4], [5], [6], [7]]
dataY : [1, 2, 3, 4, 5, 6, 7, 8]
H -> I
seq_in : I
seq_out : J
dataX : [[0], [1], [2], [3], [4], [5], [6], [7], [8]]
dataY : [1, 2, 3, 4, 5, 6, 7, 8, 9]
I -> J
seq_in : J
seq_out : K
dataX : [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
dataY : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
J -> K
seq_in : K
seq_out : L
dataX : [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9

In [5]:
print('dataX :', dataX)

dataX : [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24]]


In [6]:
print('dataY :', dataY)

dataY : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [7]:
# convert list of lists to array and pad sequences if needed

X = sequence.pad_sequences(dataX, maxlen=seq_length,dtype='float32')
print('X :', X[0:5]) # printing first 5 values

X : [[0.]
 [1.]
 [2.]
 [3.]
 [4.]]


In [8]:
# reshape X to be [samples, time steps, features]

X = np.reshape(dataX, (X.shape[0], seq_length, 1))
print('X :', X[0:5])# printing first 5 values

X : [[[0]]

 [[1]]

 [[2]]

 [[3]]

 [[4]]]


In [9]:
# normalize the input integers to the range 0-to-1, the range of
# the sigmoid activation functions used by the LSTM network

X = X/float(len(alphabet))
print('X :', X[0:5]) # printing first  5 values

X : [[[0.        ]]

 [[0.03846154]]

 [[0.07692308]]

 [[0.11538462]]

 [[0.15384615]]]


In [10]:
# One hot encode the output variables

y = utils.to_categorical(dataY)
print(' Encoded y :', y[0:5])# printing first 5 values

 Encoded y : [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [11]:
# Creating the naive LSTM model
model = Sequential()
model.add(LSTM(units=16, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# Keras shuffles the training dataset before each training epoch. To ensure the training data patterns remain sequential, we can disable this shuffling.

import datetime
start = datetime.datetime.now()

print(model.summary())
model.fit(X, y, epochs=5000, batch_size=len(dataX), verbose=1, shuffle=False)


stop = datetime.datetime.now()
print("Total time taken : ", str(stop - start))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 16)                1152      
_________________________________________________________________
dense (Dense)                (None, 26)                442       
Total params: 1,594
Trainable params: 1,594
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5000
1/1 [==============================] - 4s 4s/step - loss: 3.2551 - accuracy: 0.0400
Epoch 2/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.2547 - accuracy: 0.0400
Epoch 3/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.2544 - accuracy: 0.0400
Epoch 4/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.2541 - accuracy: 0.0400
Epoch 5/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.2537 - accuracy: 0.0400
Epoch 6/5000
1/1 [====

1/1 [==============================] - 0s 6ms/step - loss: 3.2240 - accuracy: 0.0800
Epoch 78/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.2235 - accuracy: 0.0800
Epoch 79/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.2229 - accuracy: 0.0800
Epoch 80/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.2223 - accuracy: 0.0800
Epoch 81/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.2218 - accuracy: 0.1200
Epoch 82/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.2212 - accuracy: 0.1200
Epoch 83/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.2206 - accuracy: 0.1200
Epoch 84/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.2200 - accuracy: 0.1200
Epoch 85/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.2194 - accuracy: 0.1200
Epoch 86/5000
1/1 [==============================] - 0s 8ms/step - loss: 3.2188 - accuracy: 0.1200
Epoch 87/5000
1/1 [=====

1/1 [==============================] - 0s 6ms/step - loss: 3.1539 - accuracy: 0.0800
Epoch 160/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.1527 - accuracy: 0.0800
Epoch 161/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.1515 - accuracy: 0.0800
Epoch 162/5000
1/1 [==============================] - 0s 5ms/step - loss: 3.1503 - accuracy: 0.0800
Epoch 163/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.1491 - accuracy: 0.0800
Epoch 164/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.1478 - accuracy: 0.0800
Epoch 165/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.1466 - accuracy: 0.0800
Epoch 166/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.1453 - accuracy: 0.0800
Epoch 167/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.1441 - accuracy: 0.0800
Epoch 168/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.1428 - accuracy: 0.0800
Epoch 169/5000


1/1 [==============================] - 0s 6ms/step - loss: 3.0311 - accuracy: 0.1600
Epoch 242/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.0294 - accuracy: 0.1600
Epoch 243/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.0276 - accuracy: 0.1600
Epoch 244/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.0259 - accuracy: 0.1600
Epoch 245/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.0241 - accuracy: 0.1600
Epoch 246/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.0224 - accuracy: 0.1600
Epoch 247/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.0206 - accuracy: 0.1600
Epoch 248/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.0189 - accuracy: 0.1600
Epoch 249/5000
1/1 [==============================] - 0s 7ms/step - loss: 3.0171 - accuracy: 0.1600
Epoch 250/5000
1/1 [==============================] - 0s 6ms/step - loss: 3.0153 - accuracy: 0.1600
Epoch 251/5000


Epoch 323/5000
1/1 [==============================] - 0s 6ms/step - loss: 2.8821 - accuracy: 0.2800
Epoch 324/5000
1/1 [==============================] - 0s 6ms/step - loss: 2.8803 - accuracy: 0.2800
Epoch 325/5000
1/1 [==============================] - 0s 7ms/step - loss: 2.8785 - accuracy: 0.2800
Epoch 326/5000
1/1 [==============================] - 0s 6ms/step - loss: 2.8767 - accuracy: 0.2800
Epoch 327/5000
1/1 [==============================] - 0s 5ms/step - loss: 2.8748 - accuracy: 0.2800
Epoch 328/5000
1/1 [==============================] - 0s 7ms/step - loss: 2.8730 - accuracy: 0.2800
Epoch 329/5000
1/1 [==============================] - 0s 6ms/step - loss: 2.8712 - accuracy: 0.2800
Epoch 330/5000
1/1 [==============================] - 0s 6ms/step - loss: 2.8694 - accuracy: 0.2800
Epoch 331/5000
1/1 [==============================] - 0s 6ms/step - loss: 2.8676 - accuracy: 0.2800
Epoch 332/5000
1/1 [==============================] - 0s 6ms/step - loss: 2.8659 - accuracy: 0.2800


...

1/1 [==============================] - 0s 6ms/step - loss: 1.0027 - accuracy: 1.0000
Epoch 4622/5000
1/1 [==============================] - 0s 7ms/step - loss: 1.0025 - accuracy: 1.0000
Epoch 4623/5000
1/1 [==============================] - 0s 6ms/step - loss: 1.0023 - accuracy: 1.0000
Epoch 4624/5000
1/1 [==============================] - 0s 6ms/step - loss: 1.0021 - accuracy: 1.0000
Epoch 4625/5000
1/1 [==============================] - 0s 7ms/step - loss: 1.0019 - accuracy: 1.0000
Epoch 4626/5000
1/1 [==============================] - 0s 5ms/step - loss: 1.0017 - accuracy: 1.0000
Epoch 4627/5000
1/1 [==============================] - 0s 8ms/step - loss: 1.0015 - accuracy: 1.0000
Epoch 4628/5000
1/1 [==============================] - 0s 6ms/step - loss: 1.0013 - accuracy: 1.0000
Epoch 4629/5000
1/1 [==============================] - 0s 6ms/step - loss: 1.0011 - accuracy: 1.0000
Epoch 4630/5000
1/1 [==============================] - 0s 6ms/step - loss: 1.0009 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 5ms/step - loss: 0.9867 - accuracy: 1.0000
Epoch 4703/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9865 - accuracy: 1.0000
Epoch 4704/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9863 - accuracy: 1.0000
Epoch 4705/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.9861 - accuracy: 1.0000
Epoch 4706/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9859 - accuracy: 1.0000
Epoch 4707/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.9857 - accuracy: 1.0000
Epoch 4708/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9855 - accuracy: 1.0000
Epoch 4709/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9853 - accuracy: 1.0000
Epoch 4710/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9851 - accuracy: 1.0000
Epoch 4711/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9849 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 6ms/step - loss: 0.9709 - accuracy: 1.0000
Epoch 4784/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9707 - accuracy: 1.0000
Epoch 4785/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9705 - accuracy: 1.0000
Epoch 4786/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9703 - accuracy: 1.0000
Epoch 4787/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9701 - accuracy: 1.0000
Epoch 4788/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9699 - accuracy: 1.0000
Epoch 4789/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.9697 - accuracy: 1.0000
Epoch 4790/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9695 - accuracy: 1.0000
Epoch 4791/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9693 - accuracy: 1.0000
Epoch 4792/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9691 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 7ms/step - loss: 0.9553 - accuracy: 1.0000
Epoch 4865/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9551 - accuracy: 1.0000
Epoch 4866/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9549 - accuracy: 1.0000
Epoch 4867/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9547 - accuracy: 1.0000
Epoch 4868/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9545 - accuracy: 1.0000
Epoch 4869/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9543 - accuracy: 1.0000
Epoch 4870/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9541 - accuracy: 1.0000
Epoch 4871/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9539 - accuracy: 1.0000
Epoch 4872/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.9537 - accuracy: 1.0000
Epoch 4873/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9535 - accuracy: 1.0000
Epoch 

1/1 [==============================] - 0s 6ms/step - loss: 0.9401 - accuracy: 1.0000
Epoch 4945/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9399 - accuracy: 1.0000
Epoch 4946/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9397 - accuracy: 1.0000
Epoch 4947/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9395 - accuracy: 1.0000
Epoch 4948/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9393 - accuracy: 1.0000
Epoch 4949/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.9391 - accuracy: 1.0000
Epoch 4950/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9390 - accuracy: 1.0000
Epoch 4951/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9388 - accuracy: 1.0000
Epoch 4952/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.9386 - accuracy: 1.0000
Epoch 4953/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.9384 - accuracy: 1.0000
Epoch 

In [13]:
# summarize performance of the model

scores = model.evaluate(X, y, verbose=1)
print("Model Accuracy :%.2f%%" % (scores[1]*100))

1/1 [==============================] - 1s 558ms/step - loss: 0.9294 - accuracy: 1.0000
Model Accuracy :100.00%


In [14]:
# demonstrate some model predictions

for pattern in dataX:
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)
    

['A'] -> B
['B'] -> C
['C'] -> D
['D'] -> E
['E'] -> F
['F'] -> G
['G'] -> H
['H'] -> I
['I'] -> J
['J'] -> K
['K'] -> L
['L'] -> M
['M'] -> N
['N'] -> O
['O'] -> P
['P'] -> Q
['Q'] -> R
['R'] -> S
['S'] -> T
['T'] -> U
['U'] -> V
['V'] -> W
['W'] -> X
['X'] -> Y
['Y'] -> Z


In [15]:
# demonstrate predicting random patterns
print("Test a Random Pattern :")
for i in range(0, 20):
    pattern_index = np.random.randint(len(dataX))
    pattern = dataX[pattern_index]
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

Test a Random Pattern :
['P'] -> Q
['E'] -> F
['W'] -> X
['D'] -> E
['T'] -> U
['X'] -> Y
['H'] -> I
['O'] -> P
['X'] -> Y
['I'] -> J
['O'] -> P
['K'] -> L
['I'] -> J
['H'] -> I
['G'] -> H
['E'] -> F
['Q'] -> R
['H'] -> I
['M'] -> N
['A'] -> B


##### As we expected, the network is able to use the within-sequence context to learn the alphabet, achieving 100% accuracy on the training data.

##### Importantly, the network can make accurate predictions for the next letter in the alphabet for randomly selected characters